# TypedDict vs Pydantic for Structured Output

## Complete Guide with LangChain & OpenAI

This tutorial covers:
1. TypedDict basics and usage
2. Pydantic models with validation
3. When to use each approach
4. Multiple real-world examples
5. Performance comparison

---

## Setup

In [ ]:
# Install: pip install langchain langchain-openai python-dotenv pydantic typing-extensions

In [1]:
import os
from dotenv import load_dotenv
from typing import Optional, List, Literal
from typing import TypedDict, Annotated
from pydantic import BaseModel, Field
from langchain_openai import ChatOpenAI

load_dotenv(dotenv_path="/home/bipin/Documents/genai/g25-nov-hindi/fullstack-genai-jan-26/.env")
llm = ChatOpenAI(model="gpt-4.1-mini", temperature=0)
print("✅ Setup complete!")

✅ Setup complete!


---

## Part 1: TypedDict Approach

### What is TypedDict?
- Lightweight dictionary with type hints
- No runtime validation
- Returns plain Python dict
- Best for simple schemas

### Example 1: Basic TypedDict

In [ ]:
class MovieDict(TypedDict):
    """A movie with details."""
    title: Annotated[str, ..., "The title of the movie"]
    year: Annotated[int, ..., "The year the movie was released"]
    director: Annotated[str, ..., "The director of the movie"]
    rating: Annotated[float, ..., "The movie's rating out of 10"]
    address :str
    president_name:str

movie_extractor = llm.with_structured_output(MovieDict)
response = movie_extractor.invoke("Tell me about Inception")

print("Type:", type(response))
print("Data:", response)
print(f"\nTitle: {response['title']}")
print(f"Year: {response['year']}")
print(f"Director: {response['director']}")
print(f"Rating: {response['rating']}/10")

In [ ]:


import base64
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage

# Read image file directly
with open("a1.png", "rb") as image_file:
    image_base64 = base64.b64encode(image_file.read()).decode()

model = ChatOpenAI(model="gpt-4o")

message = HumanMessage(
    content=[
        {"type": "text", "text": "Describe this image."},
        {
            "type": "image_url",
            "image_url": {"url": f"data:image/jpeg;base64,{image_base64}"}
        }
    ]
)

response = model.invoke([message])
print(response.content)

In [4]:
import base64
from typing import TypedDict, Dict, List, Any
from typing_extensions import NotRequired  # Python <3.11
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage

class LabResult(TypedDict):
    test_name: str
    date: str
    value: float
    unit: str
    ref_low: float
    ref_high: float

class KFTReport(TypedDict):
    patient_name: str
    patient_id: str
    doctor: str
    report_date: str
    lab_name: str
    results: Dict[str, LabResult]

# Encode image (same as before)
def encode_image(image_path):
    with open(image_path, "rb") as f:
        return base64.b64encode(f.read()).decode("utf-8")

image_path = "a1.png"
base64_image = encode_image(image_path)

structured_model = llm.with_structured_output(KFTReport)  # TypedDict works directly!

message = HumanMessage(
    content=[
        {
            "type": "text",
            "text": """Extract kidney function test data from this lab report image into exact TypedDict format.
            Parse ALL tests (Urea, Creatinine, eGFR, Calcium, etc.) with precise values, units, ref ranges.
            Dates in YYYY-MM-DD. Use exact table values."""
        },
        {
            "type": "image_url",
            "image_url": {"url": f"data:image/jpeg;base64,{base64_image}"},
        },
    ]
)

result: KFTReport = structured_model.invoke([message])
print(result)
# {'patient_name': 'Mohan Sharma', 'patient_id': 'MR No.: 230923003', 'doctor': 'Dr. S. K. Gupta', ...}

{'patient_name': 'Mr. Chandan Sharma', 'patient_id': 'ML02104132/3336042300013', 'doctor': 'SELF', 'report_date': '2023-04-19', 'lab_name': 'MAX Lab', 'results': {'Urea': {'value': 20.8, 'unit': 'mg/dL', 'ref_range': '17.12 - 55.64'}, 'Blood Urea Nitrogen': {'value': 9.72, 'unit': 'mg/dl', 'ref_range': '8 - 26'}, 'Creatinine': {'value': 0.78, 'unit': 'mg/dL', 'ref_range': '0.9 - 1.3'}, 'eGFR': {'value': 112.13, 'unit': 'ml/min/1.73 m²', 'ref_range': ''}, 'Bun/Creatinine Ratio': {'value': 12.46, 'unit': 'Ratio', 'ref_range': '12:1 - 20:1'}, 'Uric Acid': {'value': 15.3, 'unit': 'mg/dl', 'ref_range': '3.5 - 7.2'}, 'Calcium (Total)': {'value': 8.37, 'unit': 'mg/dl', 'ref_range': '8.9 - 10.3'}, 'Sodium': {'value': 137.4, 'unit': 'mmol/L', 'ref_range': '136 - 144'}, 'Potassium': {'value': 4.06, 'unit': 'mmol/L', 'ref_range': '3.6 - 5.1'}, 'Chloride': {'value': 109, 'unit': 'mmol/l', 'ref_range': '101-111'}, 'Bicarbonate': {'value': 21.8, 'unit': 'mmol/l', 'ref_range': '22-29'}}}


In [5]:
result

{'patient_name': 'Mr. Chandan Sharma',
 'patient_id': 'ML02104132/3336042300013',
 'doctor': 'SELF',
 'report_date': '2023-04-19',
 'lab_name': 'MAX Lab',
 'results': {'Urea': {'value': 20.8,
   'unit': 'mg/dL',
   'ref_range': '17.12 - 55.64'},
  'Blood Urea Nitrogen': {'value': 9.72,
   'unit': 'mg/dl',
   'ref_range': '8 - 26'},
  'Creatinine': {'value': 0.78, 'unit': 'mg/dL', 'ref_range': '0.9 - 1.3'},
  'eGFR': {'value': 112.13, 'unit': 'ml/min/1.73 m²', 'ref_range': ''},
  'Bun/Creatinine Ratio': {'value': 12.46,
   'unit': 'Ratio',
   'ref_range': '12:1 - 20:1'},
  'Uric Acid': {'value': 15.3, 'unit': 'mg/dl', 'ref_range': '3.5 - 7.2'},
  'Calcium (Total)': {'value': 8.37,
   'unit': 'mg/dl',
   'ref_range': '8.9 - 10.3'},
  'Sodium': {'value': 137.4, 'unit': 'mmol/L', 'ref_range': '136 - 144'},
  'Potassium': {'value': 4.06, 'unit': 'mmol/L', 'ref_range': '3.6 - 5.1'},
  'Chloride': {'value': 109, 'unit': 'mmol/l', 'ref_range': '101-111'},
  'Bicarbonate': {'value': 21.8, 'unit

In [ ]:
result

### Example 2: Product Information

In [ ]:
class ProductDict(TypedDict):
    """Product information."""
    name: Annotated[str, ..., "Product name"]
    price: Annotated[float, ..., "Price in USD"]
    category: Annotated[str, ..., "Product category"]
    in_stock: Annotated[bool, ..., "Whether product is in stock"]
    description: Annotated[str, ..., "Brief product description"]

product_extractor = llm.with_structured_output(ProductDict)
result = product_extractor.invoke(
    "Extract: Wireless Mouse - $29.99, Electronics, Available, Ergonomic design with 2.4GHz connectivity"
)

print("Product Information:")
for key, value in result.items():
    print(f"  {key}: {value}")

### Example 3: Contact Extraction

In [ ]:
class ContactDict(TypedDict, total=False):
    """Contact information (all fields optional with total=False)."""
    name: Annotated[str, "Person's full name"]
    email: Annotated[str, "Email address"]
    phone: Annotated[str, "Phone number"]
    company: Annotated[str, "Company name"]
    title: Annotated[str, "Job title"]

contact_extractor = llm.with_structured_output(ContactDict)
contact = contact_extractor.invoke(
    "Sarah Johnson, CTO at TechCorp, sarah@techcorp.com, +1-555-1234"
)

print("Extracted Contact:")
print(contact)

---

## Part 2: Pydantic Approach

### What is Pydantic?
- Full-featured data validation library
- Runtime validation
- Rich field descriptions
- Nested structures
- Custom validators

### Example 1: Basic Pydantic Model

In [ ]:
class Movie(BaseModel):
    """A movie with details."""
    title: str = Field(description="The title of the movie")
    year: int = Field(description="The year the movie was released")
    director: str = Field(description="The director of the movie")
    rating: float = Field(description="The movie's rating out of 10", ge=0, le=10)

movie_extractor_pydantic = llm.with_structured_output(Movie)
response = movie_extractor_pydantic.invoke("Tell me about Inception")

print("Type:", type(response))
print("Data:", response)
print(f"\nTitle: {response.title}")
print(f"Year: {response.year}")
print(f"Director: {response.director}")
print(f"Rating: {response.rating}/10")

### Example 2: Validation with Constraints

In [ ]:
class Product(BaseModel):
    """Product with validation."""
    name: str = Field(min_length=3, max_length=100, description="Product name")
    price: float = Field(gt=0, lt=1000000, description="Price in USD")
    category: str = Field(description="Product category")
    in_stock: bool = Field(description="Stock availability")
    quantity: int = Field(ge=0, description="Quantity available")
    discount: Optional[float] = Field(default=0, ge=0, le=100, description="Discount %")

product_extractor_pydantic = llm.with_structured_output(Product)
result = product_extractor_pydantic.invoke(
    "Laptop - $899, Electronics, In stock, 25 units, 15% off"
)

print("✅ Validation passed!")
print(f"Product: {result.name}")
print(f"Price: ${result.price}")
print(f"Stock: {result.quantity} units")
print(f"Discount: {result.discount}%")

### Example 3: Custom Validators

In [ ]:
class Email(BaseModel):
    """Email with custom validation."""
    sender: str = Field(description="Sender email address")
    subject: str = Field(description="Email subject")
    priority: Literal["high", "medium", "low"] = Field(description="Priority level")
    word_count: int = Field(description="Approximate word count")
    
    @validator('sender')
    def validate_email(cls, v):
        if '@' not in v:
            raise ValueError('Invalid email format')
        return v.lower()
    
    @validator('word_count')
    def validate_word_count(cls, v):
        if v < 0:
            raise ValueError('Word count must be positive')
        return v

email_parser = llm.with_structured_output(Email)
email = email_parser.invoke(
    "From: john@company.com, Subject: Urgent Meeting, Priority: high, approximately 150 words"
)

print(f"From: {email.sender}")
print(f"Subject: {email.subject}")
print(f"Priority: {email.priority.upper()}")
print(f"Words: ~{email.word_count}")

### Example 4: Nested Structures

In [ ]:
class Address(BaseModel):
    """Address information."""
    street: str
    city: str
    state: str
    zip_code: str

class Person(BaseModel):
    """Person with nested address."""
    name: str = Field(description="Full name")
    age: int = Field(ge=0, le=150, description="Age in years")
    email: str = Field(description="Email address")
    address: Address = Field(description="Home address")
    skills: List[str] = Field(description="List of skills")

person_parser = llm.with_structured_output(Person)
person = person_parser.invoke(
    """Alex Martinez, 28 years old, alex@email.com
    Address: 123 Main St, Portland, OR, 97201
    Skills: Python, JavaScript, Docker"""
)

print(f"Name: {person.name}, Age: {person.age}")
print(f"Email: {person.email}")
print(f"Address: {person.address.street}, {person.address.city}, {person.address.state}")
print(f"Skills: {', '.join(person.skills)}")

---

## Part 3: Real-World Examples

### Example 1: Sentiment Analysis (Pydantic)

In [ ]:
class SentimentAnalysis(BaseModel):
    """Sentiment analysis result."""
    text: str = Field(description="Original text")
    sentiment: Literal["positive", "negative", "neutral"] = Field(description="Sentiment")
    confidence: float = Field(ge=0, le=1, description="Confidence score")
    emotions: List[str] = Field(description="Detected emotions")
    key_phrases: List[str] = Field(description="Important phrases")

sentiment_analyzer = llm.with_structured_output(SentimentAnalysis)

texts = [
    "I love this product! Best purchase ever!",
    "Terrible quality, waste of money.",
    "It's okay, nothing special."
]

print("Sentiment Analysis Results:\n")
for text in texts:
    result = sentiment_analyzer.invoke(f"Analyze: {text}")
    print(f"Text: {text}")
    print(f"  Sentiment: {result.sentiment.upper()} ({result.confidence:.0%})")
    print(f"  Emotions: {', '.join(result.emotions)}")
    print()

### Example 2: Resume Parser (Pydantic)

In [ ]:
class Education(BaseModel):
    degree: str
    institution: str
    year: Optional[int] = None

class Experience(BaseModel):
    title: str
    company: str
    duration: str

class Resume(BaseModel):
    """Structured resume data."""
    name: str = Field(description="Candidate name")
    email: str = Field(description="Email")
    skills: List[str] = Field(description="Skills")
    education: List[Education] = Field(description="Education")
    experience: List[Experience] = Field(description="Work experience")

resume_parser = llm.with_structured_output(Resume)

resume_text = """
JANE DOE
jane.doe@email.com

SKILLS: Python, React, AWS, Docker

EDUCATION:
- BS Computer Science, MIT, 2018
- MS Data Science, Stanford, 2020

EXPERIENCE:
- Senior Engineer at Google (2020-Present)
- Software Engineer at Amazon (2018-2020)
"""

resume = resume_parser.invoke(f"Parse resume: {resume_text}")

print(f"Name: {resume.name}")
print(f"Email: {resume.email}")
print(f"Skills: {', '.join(resume.skills)}")
print("\nEducation:")
for edu in resume.education:
    print(f"  - {edu.degree}, {edu.institution} ({edu.year})")
print("\nExperience:")
for exp in resume.experience:
    print(f"  - {exp.title} at {exp.company} ({exp.duration})")

### Example 3: Article Classification (TypedDict)

In [ ]:
class ArticleDict(TypedDict):
    """Article metadata."""
    title: Annotated[str, "Article title"]
    category: Annotated[str, "Main category"]
    tags: Annotated[List[str], "Relevant tags"]
    summary: Annotated[str, "Brief summary"]
    reading_time: Annotated[int, "Minutes to read"]

article_classifier = llm.with_structured_output(ArticleDict)

article = """AI is transforming healthcare through early disease detection and 
personalized treatment. Machine learning algorithms analyze medical images with 
expert-level accuracy, reducing diagnosis time by 40%."""

result = article_classifier.invoke(f"Classify: {article}")

print(f"Title: {result['title']}")
print(f"Category: {result['category']}")
print(f"Tags: {', '.join(result['tags'])}")
print(f"Summary: {result['summary']}")
print(f"Reading Time: {result['reading_time']} min")

### Example 4: E-commerce Order (Pydantic)

In [ ]:
class OrderItem(BaseModel):
    product: str
    quantity: int = Field(ge=1)
    price: float = Field(gt=0)

class Order(BaseModel):
    """E-commerce order."""
    order_id: str
    customer: str
    items: List[OrderItem]
    total: float = Field(gt=0)
    status: Literal["pending", "shipped", "delivered"]

order_parser = llm.with_structured_output(Order)

order_text = """
Order #12345 for John Smith
Items: 2x Laptop ($999 each), 1x Mouse ($29)
Total: $2027
Status: Shipped
"""

order = order_parser.invoke(f"Parse: {order_text}")

print(f"Order #{order.order_id}")
print(f"Customer: {order.customer}")
print(f"Status: {order.status.upper()}")
print("\nItems:")
for item in order.items:
    print(f"  {item.quantity}x {item.product} @ ${item.price}")
print(f"\nTotal: ${order.total}")

---

## Part 4: Comparison & When to Use Each

### Side-by-Side Comparison

In [ ]:
# TypedDict version
class BookDict(TypedDict):
    title: Annotated[str, "Book title"]
    author: Annotated[str, "Author name"]
    year: Annotated[int, "Publication year"]
    pages: Annotated[int, "Number of pages"]

# Pydantic version
class BookModel(BaseModel):
    title: str = Field(description="Book title")
    author: str = Field(description="Author name")
    year: int = Field(ge=1000, le=2100, description="Publication year")
    pages: int = Field(gt=0, description="Number of pages")

# Test both
book_dict_parser = llm.with_structured_output(BookDict)
book_model_parser = llm.with_structured_output(BookModel)

query = "1984 by George Orwell, published 1949, 328 pages"

result_dict = book_dict_parser.invoke(query)
result_model = book_model_parser.invoke(query)

print("TypedDict Result:")
print(f"  Type: {type(result_dict)}")
print(f"  Data: {result_dict}")
print(f"  Access: result['title'] = {result_dict['title']}")

print("\nPydantic Result:")
print(f"  Type: {type(result_model)}")
print(f"  Data: {result_model}")
print(f"  Access: result.title = {result_model.title}")

### Decision Matrix

| Feature | TypedDict | Pydantic |
|---------|-----------|----------|
| Runtime Validation | ❌ No | ✅ Yes |
| Field Constraints | ❌ No | ✅ Yes (ge, le, gt, lt) |
| Custom Validators | ❌ No | ✅ Yes |
| Nested Models | ⚠️ Limited | ✅ Full Support |
| Return Type | dict | Pydantic Model |
| Performance | ⚡ Faster | 🐢 Slower |
| Complexity | 🟢 Simple | 🟡 More Features |
| Best For | Simple schemas | Complex validation |

### When to Use TypedDict:
✅ Simple data structures
✅ Performance-critical applications
✅ No validation needed
✅ Prefer plain dictionaries

### When to Use Pydantic:
✅ Need validation
✅ Complex nested structures
✅ Custom validators required
✅ Field constraints (min, max, etc.)
✅ Better IDE support

---

## Part 5: Advanced Features

### Including Raw Response

In [ ]:
class Summary(BaseModel):
    title: str
    summary: str
    word_count: int

model_with_raw = llm.with_structured_output(Summary, include_raw=True)
result = model_with_raw.invoke("Summarize: AI is transforming industries worldwide.")

print("Parsed Output:")
print(f"  Title: {result['parsed'].title}")
print(f"  Summary: {result['parsed'].summary}")

print("\nRaw Message:")
print(f"  Type: {type(result['raw'])}")
if hasattr(result['raw'], 'usage_metadata'):
    print(f"  Tokens: {result['raw'].usage_metadata}")

### Strict Mode (JSON Schema)

In [ ]:
class StrictData(BaseModel):
    name: str
    score: int = Field(ge=0, le=100)
    category: Literal["A", "B", "C"]

strict_model = llm.with_structured_output(
    StrictData,
    method="json_schema",
    strict=True
)

result = strict_model.invoke("Name: Test, Score: 85, Category: A")
print(f"Name: {result.name}")
print(f"Score: {result.score}")
print(f"Category: {result.category}")

---

## Summary

### Key Takeaways:

1. **TypedDict**: Lightweight, fast, returns dict, no validation
2. **Pydantic**: Full validation, constraints, nested models, custom validators
3. **Choose based on needs**: Simple → TypedDict, Complex → Pydantic
4. **Both work with** `with_structured_output()`
5. **Pydantic recommended** for most production use cases

### Best Practices:
- Use clear field descriptions
- Add constraints where appropriate
- Use Literal for fixed choices
- Leverage nested models for complex data
- Set temperature=0 for consistency

---

**Resources:**
- [LangChain Structured Output](https://docs.langchain.com/oss/python/langchain/structured-output)
- [Pydantic Docs](https://docs.pydantic.dev/)
- [TypedDict PEP](https://peps.python.org/pep-0589/)